This notebook shows an example of importing and scheduling another notebook in ModelOps. The notebook that's being scheduled is available [in artifacts directory](./artifacts/BYO_Notebook.ipynb).

# Initialize a client

First, we need to initialize a client. It will pickup ModelOps instance configuration automatically from `~/.aoa/` directory. Alternatively, you could provide connection parameters to `AoaClient` constructor.

In [16]:
from aoa import AoaClient

client = AoaClient(ssl_verify=False)

Let's add a couple of helper functions that will help us to format some API outputs, and let's see if we have a project configured.

In [17]:
import pandas as pd

pd.set_option("max_colwidth", 140)


def dict_to_frame(d, columns=None):
    if columns:
        return pd.DataFrame(
            [[k, v] for (k, v) in list(d.items()) if k in columns],
            columns=["attribute", "value"],
        )
    else:
        return pd.DataFrame(list(d.items()), columns=["attribute", "value"])


def list_to_frame(d, columns):
    return pd.DataFrame(
        [[l.get(column) for column in columns] for l in d], columns=columns
    )


list_to_frame(
    list(client.projects()),
    ["id", "name", "description", "groupId", "gitRepositoryUrl", "createdAt"],
)

,id,name,description,groupId,gitRepositoryUrl,createdAt
0,70d4659b-92a2-4723-841a-9ba5629b5f27,Demo,Example ML Use Cases,DEMO,/app/built-in/demo-models,2019-01-13T21:04:06.112+00:00
1,23e1df4b-b630-47a1-ab80-7ad5385fcd8d,Test,Test Use Cases,DEMO,https://github.com/teradata/modelops-demo-models.git,2019-01-13T21:04:06.112+00:00


Next, we'll configure a current project for our client instance. It will allow us to skip project ID for all other calls we make.

In [18]:
client.set_project_id("70d4659b-92a2-4723-841a-9ba5629b5f27")

Now we can ask ModelOps to describe this project.

In [19]:
client.describe_current_project()

,attribute,value
0,name,Demo
1,id,70d4659b-92a2-4723-841a-9ba5629b5f27
2,description,Example ML Use Cases
3,groupId,DEMO
4,gitRepositoryUrl,/app/built-in/demo-models
5,branch,tmo
6,gitCredentials,{'source': 'environment'}
7,createdAt,2019-01-13T21:04:06.112+00:00
8,archived,False
9,gitStatus,"{'healthy': True, 'cloneHealthy': True, 'pullHealthy': True, 'lastPullSuccessDate': '2024-11-08T08:35:27.768+00:00', 'monitored': True}"


# Define BYOM model and import model version (the notebook itself)

First, we should define what we want to import:
1. The notebook itself
2. Optionally, `requirements.txt` file for any additional Python requirements
3. Any additional artifacts

In [20]:
files = ["./artifacts/BYO_Notebook.ipynb", "./artifacts/requirements.txt"]
language = "Jupyter"

Next, let's define a model in ModelOps, under which we'll register this notebook as a new model version. Alternatively, you could use an existing model, make sure its ID is in `model` variable.

In [21]:
import uuid

import_id = str(uuid.uuid4())

model_dict = {
    "name": f"{language}_Demo_{uuid.uuid4().clock_seq}",
    "description": f"{language} model defined from Python SDK",
    "language": language,
}
model_response = client.models().save(model_dict)
model = model_response["id"]

Next, we upload desired artifacts. Notice that we should preserve `import_id` for additional imports and to instruct ModelOps to registed imported artifacts as BYOM model version.

In [22]:
client.trained_model_artefacts().upload_artefacts(import_id, artefacts=files)

Now, we can register this imported artifact as a BYOM model version. `import_request` here is very simple:
- `artefactImportId` is that `import_id` we've used before to upload artifacts
- `externalId` is any non-empty string that you could use to track this particular artifact (e.g. Notebook identifier in a different system)

In [23]:
import_request = {"artefactImportId": import_id, "externalId": str(uuid.uuid4())}
import_job = client.models().import_byom(model, import_request)
imported_model_id = import_job["metadata"]["trainedModel"]["id"]
client.jobs().wait(import_job["id"])
print("Model imported")

Model imported


# Approve imported model version (notebook) for production deployment

The next step is simple, but important. We have to approve the model version in order to allow us (and others) to deploy it. We need just a model version ID, and we are free to provide any comment for the approval.

In [24]:
dict_to_frame(client.trained_models().approve(imported_model_id, comments="LGTM"))

,attribute,value
0,id,474f25da-7f98-4964-85c3-d1522996ed13
1,projectId,70d4659b-92a2-4723-841a-9ba5629b5f27
2,ownerId,admin
3,createdAt,2024-11-08T08:35:44.953+00:00
4,trainedModelId,2aaad0c4-4f56-4833-be02-a6de04560ba4
5,status,APPROVED
6,metadata,{'comments': 'LGTM'}


# Schedule a notebook for single execution

Once the model version is approved, we can schedule it for regular (or singular) execution. First, we need a `dataset_template` identifier, that will be provided as metadata to our notebook.

In [25]:
dataset_template = client.dataset_templates().find_by_name_like("PIMA")["_embedded"][
    "datasetTemplates"
][0]
dict_to_frame(dataset_template, columns=["id", "name", "description"])

,attribute,value
0,id,4053c309-2811-4f03-9fb4-e99c728ecdfb
1,name,PIMA Diabetes
2,description,Vantage PIMA diabetes Dataset Template


Next, we define a deployment request and use it to deploy the model version.

In [26]:
deploy_request = {
    "engineType": "DOCKER_BATCH",
    "engineTypeConfig": {
        "dockerImage": "artifacts.td.teradata.com/tdproduct-docker-snapshot/avmo/vmo-python-base:3.11.4",
        "engine": "jupyter",
        "resources": {
            "memory": "2g",
            "cpu": "1",
        },
    },
    "language": "jupyter",
    "datasetConnectionId": client.get_default_connection_id(),
    "byomModelLocation": {"database": "trng_modelops", "table": "aoa_byom_models"},
    "datasetTemplateId": dataset_template["id"],
    "cron": "@once",
    "publishOnly": "false",
    "customProperties": {},
}

deploy_job = client.trained_models().deploy(imported_model_id, deploy_request)

client.jobs().wait(deploy_job["id"])
print("Model deployed")

Model deployed


# Monitor the prediction job

Once the model version is deployed, we should start looking for prediction jobs initiated by this deployment:

In [27]:
deployment = client.deployments().find_by_deployment_job_id(deploy_job["id"])
jobs = client.jobs().find_by_deployment_id(deployment["id"], "expandJob")["_embedded"][
    "jobs"
]
list_to_frame(jobs, ["id", "type", "status", "createdAt", "updatedAt"])

,id,type,status,createdAt,updatedAt
0,83b8a2bf-a8b5-459b-8d97-d22ecbfc5329,BATCH_PREDICTION,ASSIGNED,2024-11-08T08:35:45.131+00:00,2024-11-08T08:35:45.680+00:00


Looks like there's at least a single job, let's monitor it:

In [28]:
if len(jobs) == 1:
    client.jobs().wait(jobs[0]["id"])
    print("Job completed")
elif len(jobs) == 0:
    print("No jobs found")
else:
    print("Multiple jobs found")

Job completed
